In [1]:
# !pip install transformers accelerate
from transformers import AutoTokenizer, AutoModel, EarlyStoppingCallback, AutoModelForSequenceClassification, AutoConfig,Trainer, TrainingArguments,DataCollatorWithPadding
import torch
import numpy as np
import pandas as pd
# !pip install datasets
from datasets import load_metric
# !pip install evaluate
from evaluate import evaluator
from sklearn.model_selection import train_test_split
import csv

# notebook_login()

CUDA_LAUNCH_BLOCKING=1


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv(r'/content/drive/MyDrive/train_50.csv', encoding='utf_8_sig')
test = pd.read_csv(r'/content/drive/MyDrive/test_50.csv', encoding='utf_8_sig')


In [4]:
train

,label,text,diff
0,negative,Fix #2197 extended regex pattern example,diff --git a/docs/content/manual/manual.yml b/...
1,negative,some test spec fixes,diff --git a/asset-pipeline-grails/src/test/gr...
2,positive,"FILEUPLOAD-279: Introduce a system property, w...",diff --git a/.gitignore b/.gitignore\nnew file...
3,negative,fix(documentation): remove route redeclaration...,diff --git a/doc/API/centreon-api-v2.yaml b/do...
4,negative,[JENKINS-49744] - Generalize the permission ha...,diff --git a/src/main/java/com/synopsys/arc/je...
...,...,...,...
96,negative,[maven-release-plugin] prepare release checkst...,diff --git a/pom.xml b/pom.xml\nindex 2043823....
97,negative,Merge pull request #360 from joaosantos81/mast...,diff --git a/control/Session.php b/control/Ses...
98,negative,Repairing: implement invalid node checksum fix.,diff --git a/libexfat/exfat.h b/libexfat/exfat...
99,positive,[MJ2] Avoid index out of bounds access to pi->...,diff --git a/src/lib/openmj2/pi.c b/src/lib/op...


In [5]:
len(train)

101

In [6]:
train['label'].value_counts()

negative    63
positive    38
Name: label, dtype: int64

In [7]:
test['label'].value_counts()

negative    6284
positive    3727
Name: label, dtype: int64

In [8]:
# !pip install sentence_transformers
from sentence_transformers import SentenceTransformer
model = SentenceTransformer('paraphrase-mpnet-base-v2')

In [9]:
train['embeddings'] = train['text'].apply(lambda x: model.encode(str(x)))
test['embeddings'] = test['text'].apply(lambda x: model.encode(str(x)))

In [10]:
test

,label,text,diff,embeddings
0,negative,Merge pull request #1015 from omniauth/make-su...,diff --git a/Gemfile b/Gemfile\nindex b0581c01...,"[-0.1300403, 0.12006516, -0.08801293, 0.104843..."
1,negative,"Cleaned up rdpei channel, updated to current s...",diff --git a/channels/rdpei/client/rdpei_main....,"[-0.04262921, -0.061168917, 0.026884632, -0.02..."
2,negative,Make remaining AST classes public. Fixes jkuhn...,diff --git a/src/main/java/ognl/ASTAdd.java b/...,"[0.045945365, 0.17283365, -0.029472586, 0.0503..."
3,negative,Test custom field uploads on ticket create for...,diff --git a/t/rest2/tickets.t b/t/rest2/ticke...,"[-0.09309566, 0.22370622, -0.043404777, 0.1327..."
4,negative,Merge branch '2.7' into master,diff --git a/.travis.yml b/.travis.yml\nindex ...,"[-0.117159605, -0.009933155, 0.051275827, -0.0..."
...,...,...,...,...
10006,positive,Also unfold and split lazily\n\nThis has littl...,diff --git a/lib/mail/field.rb b/lib/mail/fiel...,"[-0.0686344, -0.053742427, -0.08566993, 0.1238..."
10007,negative,Adding a property 'jdk.xml.resolveExternalEnti...,diff --git a/src/org/apache/xerces/parsers/Sec...,"[-0.037466474, 0.21927895, -0.028538624, 0.022..."
10008,positive,name might not be displayed correctly\n\nChang...,diff --git a/tine20/Addressbook/js/ContactGrid...,"[-0.049099416, 0.19766138, -0.05566096, 0.1250..."
10009,negative,Trailing whitespace...\n\nMake Travis happy,diff --git a/airsonic-main/src/main/java/org/a...,"[-0.02027782, 0.023705442, -0.14176506, 0.1491..."


In [11]:
X_train = train['embeddings'].to_list()
X_test = test['embeddings'].to_list()

y_train = train['label'].to_list()
y_test = test['label'].to_list()

In [12]:
from sklearn.linear_model import LogisticRegression
LR = LogisticRegression()
LR.fit(X_train,y_train)

LogisticRegression()

In [13]:
from sklearn import metrics
predicted = LR.predict(X_test)
print("Logistic Regression Accuracy:",metrics.accuracy_score(y_test, predicted))
print("Logistic Regression Precision:",metrics.precision_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.recall_score(y_test, predicted,average='weighted'))
print("Logistic Regression Recall:",metrics.f1_score(y_test, predicted,average='weighted'))

Logistic Regression Accuracy: 0.7833383278393767
Logistic Regression Precision: 0.780282647816306
Logistic Regression Recall: 0.7833383278393767
Logistic Regression Recall: 0.7800807003527102
